# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

- 17 marketing campaigns.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [153]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [53]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

- The objective is to predict how many customers will subscribe to a term deposit based on their demographic, financial, and frequency of contact information.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [69]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [68]:
# Ordinal Encoder for features that can be ordered naturally
trf = ColumnTransformer([
    ('m', OrdinalEncoder(categories=[['unknown', 'single', 'divorced', 'married']]), ['marital']),
    ('ed', OrdinalEncoder(categories=[['unknown', 'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'professional.course', 'university.degree']]), ['education']),
    ('dhl', OrdinalEncoder(categories=[['unknown', 'no', 'yes'], ['unknown', 'no', 'yes'], ['unknown', 'no', 'yes']]), ['default', 'housing', 'loan']),
    ('j', OneHotEncoder(sparse_output=False), ['job'])],
    remainder='passthrough',
    verbose_feature_names_out=False
)
keep = df[['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']]
data = trf.fit_transform(keep)
data = pd.DataFrame(data, columns=trf.get_feature_names_out())
data

,marital,education,default,housing,loan,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,age
0,3.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0
1,3.0,5.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,57.0
2,3.0,5.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,37.0
3,3.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0
4,3.0,5.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,3.0,6.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0
41184,3.0,6.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0
41185,3.0,7.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0
41186,3.0,6.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,44.0


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [70]:
X_train, X_test, y_train, y_test = train_test_split(data, df[['y']], shuffle=True, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [71]:
from sklearn.dummy import DummyClassifier

In [73]:
d = DummyClassifier()
d.fit(X_train, y_train)
preds = d.predict(X_test)

In [88]:
print("Baseline score:", d.score(X_test, y_test))

Baseline score: 0.8880256385354958


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [154]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
lrpipe = Pipeline([
    ('sc', StandardScaler()),
    ('lr', LogisticRegression())
])
lrtime = %timeit -q -o lrpipe.fit(X_train, y_train)
lr_pred = lrpipe.predict(X_test)

### Problem 9: Score the Model

What is the accuracy of your model?

In [100]:
lr_train = accuracy_score(y_train, lrpipe.predict(X_train))
lr_test = accuracy_score(y_test, lr_pred)
print("Logistic Regression accuracy:", lr_test)

Logistic Regression accuracy: 0.8880256385354958


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [124]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
knn = KNeighborsClassifier()
knntime = %timeit -q -o knn.fit(X_train, y_train)

In [105]:
knn_train = accuracy_score(y_train, knn.predict(X_train))
knn_test = accuracy_score(y_test, knn.predict(X_test))

In [106]:
dt = DecisionTreeClassifier()
dttime = %timeit -q -o dt.fit(X_train, y_train)

In [114]:
dt_train = accuracy_score(y_train, dt.predict(X_train))
dt_test = accuracy_score(y_test, dt.predict(X_test))

In [ ]:
svm = SVC()
svmtime = %timeit -q -o svm.fit(X_train, y_train)

In [109]:
svm_train = accuracy_score(y_train, svm.predict(X_train))
svm_test = accuracy_score(y_test, svm.predict(X_test))

In [123]:
l = pd.DataFrame({"Model": "Logistic Regression", "Train Time": lrtime, "Train Accuracy": lr_train, "Test Accuracy": lr_test}, index=[0])
k = pd.DataFrame({"Model": "KNN", "Train Time": knntime, "Train Accuracy": knn_train, "Test Accuracy": knn_test}, index=[0])
d = pd.DataFrame({"Model": "Decision Tree", "Train Time": dttime, "Train Accuracy": dt_train, "Test Accuracy": dt_test}, index=[0])
s = pd.DataFrame({"Model": "SVM", "Train Time": svmtime, "Train Accuracy": svm_train, "Test Accuracy": svm_test}, index=[0])
a = pd.concat([l, k, d, s], ignore_index=True)
a

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,107 ms ± 13.3 ms per loop (mean ± std. dev. of...,0.887119,0.888026
1,KNN,32.3 ms ± 1.61 ms per loop (mean ± std. dev. o...,0.889871,0.875983
2,Decision Tree,153 ms ± 8.26 ms per loop (mean ± std. dev. of...,0.916966,0.864912
3,SVM,10.8 s ± 258 ms per loop (mean ± std. dev. of ...,0.887119,0.888026


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [127]:
# Logistic Regression
lr_params = {"C": np.linspace(0.0001, 0.5), "multi_class": ['ovr', 'multinomial']}
sc = StandardScaler()
scaled = pd.DataFrame(sc.fit_transform(data), columns=sc.get_feature_names_out())
lrgrid = GridSearchCV(LogisticRegression(), lr_params, cv=5)
lrgrid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 1.03020408e-02, 2.05040816e-02, 3.07061224e-02,
       4.09081633e-02, 5.11102041e-02, 6.13122449e-02, 7.15142857e-02,
       8.17163265e-02, 9.19183673e-02, 1.02120408e-01, 1.12322449e-01,
       1.22524490e-01, 1.32726531e-01, 1.42928571e-01, 1.53130612e-01,
       1.63332653e-01, 1.73534694e-01, 1.837367...
       2.85757143e-01, 2.95959184e-01, 3.06161224e-01, 3.16363265e-01,
       3.26565306e-01, 3.36767347e-01, 3.46969388e-01, 3.57171429e-01,
       3.67373469e-01, 3.77575510e-01, 3.87777551e-01, 3.97979592e-01,
       4.08181633e-01, 4.18383673e-01, 4.28585714e-01, 4.38787755e-01,
       4.48989796e-01, 4.59191837e-01, 4.69393878e-01, 4.79595918e-01,
       4.89797959e-01, 5.00000000e-01]),
                         'multi_class': ['ovr', 'multinomial']})

In [129]:
lrbest = lrgrid.best_estimator_
lr_train = accuracy_score(y_train, lrbest.predict(X_train))
lr_test = accuracy_score(y_test, lrbest.predict(X_test))

In [131]:
# KNN
k_params = {"n_neighbors": range(1, len(X_train)//500)}
kgrid = GridSearchCV(KNeighborsClassifier(), k_params, cv=5)
kgrid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 61)})

In [132]:
kbest = kgrid.best_estimator_
knn_train = accuracy_score(y_train, kbest.predict(X_train))
knn_test = accuracy_score(y_test, kbest.predict(X_test))

In [138]:
# Decision Tree
dt_params = {"criterion": ['gini', 'entropy'], "max_depth": range(1, 10), "min_samples_split": range(1, 5), "min_samples_leaf": range(1, 5), "min_impurity_decrease": np.linspace(0.01, 0.5, 5)}
dtgrid = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=5)
dtgrid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 10),
                         'min_impurity_decrease': array([0.01  , 0.1325, 0.255 , 0.3775, 0.5   ]),
                         'min_samples_leaf': range(1, 5),
                         'min_samples_split': range(1, 5)})

In [139]:
dtbest = dtgrid.best_estimator_
dt_train = accuracy_score(y_train, dtbest.predict(X_train))
dt_test = accuracy_score(y_test, dtbest.predict(X_test))

In [148]:
svm_params = {"kernel": ['linear', 'rbf', 'sigmoid'], "coef0": [1]}
svmgrid = GridSearchCV(SVC(), svm_params, cv=3)
svmgrid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'coef0': [1], 'kernel': ['linear', 'rbf', 'sigmoid']})

In [155]:
# SVM
sbest = svmgrid.best_estimator_
svm_train = accuracy_score(y_train, sbest.predict(X_train))
svm_test = accuracy_score(y_test, sbest.predict(X_test))

In [152]:
final = a[['Model', 'Train Accuracy', 'Test Accuracy']]
final['Final Training Accuracy'] = [lr_train, knn_train, dt_train, svm_train]
final['Final Testing Accuracy'] = [lr_test, knn_test, dt_test, svm_test]
final

,Model,Train Accuracy,Test Accuracy,Final Training Accuracy,Final Testing Accuracy
0,Logistic Regression,0.887119,0.888026,0.887119,0.888026
1,KNN,0.889871,0.875983,0.887702,0.887152
2,Decision Tree,0.916966,0.864912,0.887119,0.888026
3,SVM,0.887119,0.888026,0.887119,0.888026


## Findings

Categorical data had to be encoded into numeric data.

While training on an SVM model, the data was found to be linearly separable.

On the default settings of the models, Decision Trees had the highest training accuracy while Logistic Regression and SVMs had the highest testing accuracy. Interestingly, the testing accuracy for Logistic Regression and SVMs were the same as the accuracy of the baseline classifier.

Upon hyperparameter tuning for each of the four models, KNN had the best training accuracy by a small margin, while the rest of the three models had comparable testing accuracy at the end. KNN and Decision Trees showed improvement on their testing accuracies upon hyperparameter tuning.

### Limitations

Only the initial bank information was considered. Many hyperparameters had to be reduced or removed altogether due to time and computational limitations.

### Next Steps

Since all of the models reached similar levels of accuracy, more features are needed to make more informed decisions about the data. The omitted or reduced hyperparameters could be considered on a machine with higher computational power and on a larger timeframe.

Further probing into the data and the models can reveal the true correlations behind each feature and the target variable. For now, any of the four models can be recommended to a business client who wishes to predict whether a customer will subscribe to a term deposit, given that the models are tuned.